We show how to compute the symmetrized product of two symmetric matrices $A$, $B$, defined by $C=AB+BA$ using ${n+2 \choose 3}$ multiplications rather than $n^3$. Some third order intermediate tensors are computed using the Cyclops Tensor Framework (CTF) library

In [1]:
import numpy as np
import ctf

Generate random symmetric matrices and compute a reference answer using `numpy`.

In [2]:
n=4
def get_rand_sym_mat(n):
    A = np.random.random((n,n))-.5
    A = np.triu(A)
    A = A + A.T
    return A
A = get_rand_sym_mat(n)
B = get_rand_sym_mat(n)

C_ref = A @ B + B @ A

Compute intermediate tensor $$\hat{Z}_{ijk} = (A_{ij}+A_{ik}+A_{jk})\cdot(B_{ij}+B_{ik}+B_{jk})$$

In [3]:
A = ctf.astensor(A)
B = ctf.astensor(B)
C = ctf.tsr([n,n])

Zhat = ctf.tsr([n,n,n])
Zhat.i("ijk")<<(A.i("ij")+A.i("ik")+A.i("jk"))*(B.i("ij")+B.i("ik")+B.i("jk"))
A

array([[-0.7012935 ,  0.25441344,  0.43707546, -0.30600761],
       [ 0.25441344,  0.01987776, -0.43530173, -0.17038102],
       [ 0.43707546, -0.43530173, -0.39342573,  0.17131543],
       [-0.30600761, -0.17038102,  0.17131543, -0.8083512 ]])

Write a function to verify symmetry of tensors and apply it to $\hat{Z}$. We verify symmetry by checking that any transposition of modes leads to the same tensor. For a third order tensor, this implies that it has ${n+2\choose 3}$ unique elements. We will avoid accessing the diagonals of $\hat{Z}$, meaning they do not have to be computed, and only ${n\choose 3}$ elements of $\hat{Z}$ will be computed.

In [4]:
def verify_sym(A):
    for i in range(A.ndim):
        for j in range(i):
            istr1 = np.asarray(range(A.ndim))
            istr2 = istr1.copy()
            istr2[i] = istr1[j]
            istr2[j] = istr1[i]
            str1 = ''.join(str(s) for s in istr1)
            str2 = ''.join(str(s) for s in istr2)
            B = ctf.tsr(A.shape)
            B.i(str1) << A.i(str1)
            B.i(str2) << -1.0*B.i(str1)
            assert(B.norm2()/B.size < .000001)
#verify symmetry
verify_sym(A)
verify_sym(Zhat)
#zero out diagonal elements we will not use
Zhat.i("jii").scl(0.0) 
Zhat.i("iji").scl(0.0) 
Zhat.i("iij").scl(0.0)
#print(Zhat)
print("Number of unique nonzeros in Zhat is now", np.count_nonzero(Zhat.to_nparray())/6)
print("This is also the number of multiplications we need to compute Zhat")

Number of unique nonzeros in Zhat is now 4.0
This is also the number of multiplications we need to compute Zhat


Now compute a partial sum of $\hat{Z}$ into $Z$.

In [5]:
Z = ctf.tsr(A.shape)
Z.i("ij") << Zhat.i("ijk")
Z

array([[ 0.        , -0.08308564, -0.11006679,  0.06560533],
       [-0.08308564,  0.        ,  0.09919776,  0.27486988],
       [-0.11006679,  0.09919776,  0.        ,  0.24788873],
       [ 0.06560533,  0.27486988,  0.24788873,  0.        ]])

We define a function `a = lapsum(A)` which computes
$$a_i = A_{ii} - \sum_{j\neq i} A_{ij}$$

In [6]:
def lapsum(A):
    B = ctf.tsr([n])
    B.i("i") << -1.0*A.i("ij") + A.i("ii")
    return B
a = lapsum(A)
b = lapsum(B)
def nplapsum(A):
    B = np.zeros(n);
    B[:] -= np.sum(A[:,i] for i in range(n))
    B += 2.*np.diagonal(A)
    return B
print(lapsum(A))
#print(nplapsum(A.to_nparray()))
print(nplapsum(A.to_nparray()) @ nplapsum(B.to_nparray())-np.sum(nplapsum(A.to_nparray()*nplapsum(B.to_nparray()))))
#print(nplapsum(A.to_nparray()) @ nplapsum(B.to_nparray())-np.sum(nplapsum(A.to_nparray()*B.to_nparray())))
#print(A)
#a

array([-0.38548128,  0.35126932, -0.17308915,  0.30507321])
0.0


Compute matrix
$$U_{ij} = A_{ij}\cdot \left(-\frac{n}{2}B_{ij} + b_i + b_j\right)+\left(-\frac{n}{2}A_{ij} + a_i + a_j\right)\cdot B_{ij} $$
where `a = lapsum(A)` and `b = lapsum(B)`. The elements of $U$ are meant to cancel some unwated terms in corresponding element of $Z$. Only $2{n \choose 2}$ multiplications are needed to compute $U$, since both terms in it are symmetric and we will not use its diagonal.

In [7]:
U = ctf.tsr(A.shape)
U.i("ij") <<           (-(n/2.)*A.i("ij") + a.i("i") + a.i("j"))*B.i("ij")
U.i("ij") << A.i("ij")*(-(n/2.)*B.i("ij") + b.i("i") + b.i("j"))
#ignore diagonal
U.i("ii").scl(0.0)
U

array([[ 0.        ,  0.65200367, -0.13026307,  0.2364584 ],
       [ 0.65200367,  0.        , -0.82856412, -0.13424194],
       [-0.13026307, -0.82856412, -0.        ,  0.25717807],
       [ 0.2364584 , -0.13424194,  0.25717807,  0.        ]])

We know compute the vector, using the last $n$ multiplications (putting the total to ${n+2 \choose 3}$)
$$w_i = \left(\sum_j A_{ij}\right)\cdot \left(\sum_j B_{ij}\right)$$

In [8]:
w = ctf.tsr([n])
w.i("i") << ctf.sum(A,axis=0).i("i")*ctf.sum(B,axis=0).i("i")
w

array([-0.10997207, -0.01973192,  0.21302036, -0.70954666])

In [15]:
X = ctf.tsr(A.shape)
X.i("ij") << Z.i("ij")
X.i("ij") << U.i("ij")
X.i("ij") << (1./n)*U.i("ik") #!
X.i("ij") << (1./n)*U.i("jk") #!
X.i("ij") << (1./n)*w.i("i")
X.i("ij") << (1./n)*w.i("j")
#X.i("ij") << 4.*A.i("ij")*B.i("ij")

print(X)
S = ctf.tsr(A.shape)
S.i("ij") << A.i("ij")*B.i("ij")
print(S)
C_ans = ctf.astensor(C_ref)
Y = ctf.tsr(A.shape)
Y.i("ij") << C_ans.i("ij")
Y.i("ij") << (1./n)*C_ans.i("ik")
Y.i("ij") << (1./n)*C_ans.i("jk")
print(Y)

array([[ 0.32411346,  0.64834118, -0.20043032,  0.37658242],
       [ 0.64834118, -0.16526716, -0.93415712, -0.02954367],
       [-0.20043032, -0.93415712, -0.24431438,  0.29537158],
       [ 0.37658242, -0.02954367,  0.29537158, -0.17507606]])
array([[-0.2874998 , -0.11379012,  0.06121276, -0.07511788],
       [-0.11379012,  0.01414053,  0.08610409,  0.00115246],
       [ 0.06121276,  0.08610409,  0.23109086, -0.05510561],
       [-0.07511788,  0.00115246, -0.05510561, -0.58218565]])
array([[-1.3964173 ,  0.20953399,  0.05215405, -1.44249479],
       [ 0.20953399,  0.15454779,  0.28454588, -0.2444073 ],
       [ 0.05215405,  0.28454588,  1.20895566,  0.39063512],
       [-1.44249479, -0.2444073 ,  0.39063512, -2.22440174]])


Now, lastly we need a vector of the form $$w_{i}=\sum_{j\neq i} A_{ij}\cdot B_{ij}$$ to cancel the remaining terms appearing in $Z_{ij}-U_{ij}+V_{ij}$ that we do not want, i.e. we want $w_{i} + w_{j} = Z_{ij} - U_{ij} + V_{ij}$. We could compute $w$ directly via $n \choose 2$ multiplications. Instead, we get it from $U$ and $V$ via only $n$ multiplications, as we can show that 
$$\sum_{j\neq i} U_{ij}-V_{ij} = w_i + \sum_{j\neq i} \Big[A_{ij}(b_i + b_j) + (a_i+a_j)B_{ij}\Big].$$
We can simplify the latter unwanted term as
$$\sum_{j\neq i} (A_{ii}-a_{i})b_i + A_{ij}b_j + a_i(B_{ii}-b_i)+ a_jB_{ij}$$
Applying the identity $<j(A)j(B)> = \sum j(A\odot j(B))$, we get (FIXME: things don't quite cancel, adjust scaling in j(A))
$$w_i - \sum_{j\neq i} U_{ij}-V_{ij} = (n-1)(A_{ii}b_i + a_iB_{ii})= (n-1)(A_{ii}+a_i)(b_i+B_{ii}) - (n-1)A_{ii}B_{ii}.$$
Therefore we can compute
$$w_i + frac{n-1}2(U_{ii}-V_{ii})-\sum_{j\neq i} (U_{ij}-V_{ij})=\frac{n-1}2(2A_{ii}b_i+2B_{ii}a_i) - ((n-1)(A_{ii}+a_i)(b_i+B_{ii})-  (n-1)A_{ii}B_{ii}) =  (n-1)(A_{ii}+a_i)(b_i+B_{ii}).$$
We then need only to compute $n$ multiplications $(A_{ii}+a_i)(b_i+B_{ii})$.

In [10]:
Vref = ctf.tsr([n])
V = ctf.tsr([n])
Vref.i("i") << A.i("ij")*B.i("ij")
X = ctf.tsr([n,n])
X.i("ij") << Z.i("ij") +- 2.*U.i("ij") + 2.*V.i("ij")
#print(X)
V.i("i") << X.i("ij")
#V.i("i") << U.i("ij")+-1.0*V.i("ij")
#V.i("i") << -(n-1.)*(A.i("ii")+a.i("i"))*(B.i("ii")+b.i("i"))
print(V)
print(Vref)

array([-1.64394509,  0.91258678,  1.64031792, -0.13042512])
array([-0.41519503, -0.01239304,  0.3233021 , -0.71125667])
